# Run lephare

In this notebook we will take an output catalogue on a single patch, merge in the spectroscopic redshift and run lephare. the aim is to produce a general code that can be parellelised across the entire field.

We broadly follow the example run given on the main page for the new c++ version:

https://gitlab.lam.fr/Galaxies/LEPHARE

Broadly speaking we need a .para file which specifies the config for the run and an input file with all the measurements and errors and spectroscopic redshift.

In [189]:
from astropy.table import Table
from pathlib import Path
from herschelhelp_internal.masterlist import merge_catalogues, nb_merge_dist_plot, specz_merge
from herschelhelp_internal.utils import coords_to_hpidx, ebv, gen_help_id, inMoc, mag_to_flux
import astropy.units as u
import numpy as np
import time

In [21]:
!pwd

/Users/raphaelshirley/Documents/github/lsst-ir-fusion/dmu6/dmu6_VIDEO


In [24]:
#An example input:
Table.read('/Users/raphaelshirley/Documents/github/LEPHARE/examples/COSMOS.in',format='ascii')[:5]

Id,U,dU,B,dB,V,dV,R,dR,I,dI,Zn,dZn,Yhsc,dYhsc,Y,dY,J,dJ,H,dH,K,dK,Hw,dHw,Kw,dKw,IA427,dIA427,IA464,dIA464,IA484,dIA484,IA505,dIA505,IA527,dIA527,IA574,dIA574,IA624,dIA624,IA679,dIA679,IA709,dIA709,IA738,dIA738,IA767,dIA767,IA827,dIA827,NB711,dNB711,NB816,dNB816,ch1,dch1,ch2,dch2,ch3,dch3,ch4,dch4,NUV,dNUV,bidon,bidon_1,ucfhtls,ducfhtls,gcfhtls,dgcfhtls,rcfhtls,drcfhtls,icfhtls,dicfhtls,ycfhtls,dycfhtls,zcfhtls,dzcfhtls,context,z-spec
int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,float64
1,1.5763660341703e-28,7.26666089702958e-31,2.65457114182598e-28,2.53381502067042e-30,9.3144854758703e-28,6.3120512003925e-30,1.15991960470081e-27,5.1218735350011e-30,1.94864935266787e-27,4.9161461594729e-30,3.07852960829942e-27,1.58787538020643e-30,3.38596707299833e-27,5.2804246032907e-30,-99.9,-99.9,-99.9,-99.9,-99.9,-99.9,-99.9,-99.9,7.23514663587563e-27,7.2847065245261e-30,8.69713321769458e-27,7.99574187287666e-30,2.18332932007632e-28,1.38611924314223e-30,3.39742729486782e-28,1.71450634183319e-30,4.82749531662162e-28,1.82010205948151e-30,6.23712376909149e-28,1.99556925450414e-30,6.80774925965078e-28,1.88399461646408e-30,9.62584362157748e-28,3.13902124217789e-30,1.14866639589654e-27,2.48584490574606e-30,1.70177264708706e-27,4.60997668338033e-30,1.64802241613819e-27,3.92426433034385e-30,1.83884622524075e-27,3.10920711017425e-30,1.96257131499692e-27,3.3985707713843e-30,2.32250470224367e-27,3.93245948134509e-30,1.5912127000877e-27,2.32004752571559e-30,2.29509600655104e-27,3.92044883983326e-30,4.24471648490778e-27,2.57575337343852e-29,3.59655796363041e-27,2.02678957066465e-29,2.42605029441076e-27,1.20096191369726e-28,7.25228862936138e-27,1.72291939813558e-28,4.41147783235251e-29,3.65072961041126e-30,-99.9,-99.9,-99.9,-99.9,-99.9,-99.9,-99.9,-99.9,-99.9,-99.9,-99.9,-99.9,-99.9,-99.9,4294965375,0.2061
2,7.59655425823231e-28,1.87738109500118e-30,-99.9,-99.9,5.24453395190441e-27,2.3872530715425e-29,7.33355549689458e-27,2.0086585308145e-29,7.49825016265711e-27,1.52561826230575e-29,1.70933812789214e-26,9.68952982256966e-30,1.94193705081067e-26,1.9953445270225e-29,1.94029066600629e-26,4.47230008383894e-30,2.48070221072767e-26,6.77483832354773e-30,2.92354023776328e-26,9.53428405354708e-30,2.96801165873156e-26,8.61226843456251e-30,3.44183102931206e-26,2.06500401567258e-29,3.05340140121135e-26,2.60221953873401e-29,1.21238189696316e-27,4.42918617959154e-30,2.49205229614748e-27,6.24411196200976e-30,3.32080768460673e-27,7.04095699376858e-30,4.14545436603905e-27,7.26529200757928e-30,4.72860890011052e-27,7.50700335851632e-30,5.67000546468386e-27,1.13461676224691e-29,7.79525232849656e-27,9.8671764226863e-30,1.09460727039906e-26,1.77353730721346e-29,1.00392128873394e-26,1.48960492952605e-29,1.11455508100569e-26,1.15550238824338e-29,1.09068390837471e-26,1.23518742748015e-29,1.31206176094453e-26,1.52044832860116e-29,1.00124008158424e-26,6.73621348034618e-30,1.29761485982004e-26,1.36546986447873e-29,1.4213580582554e-26,3.10185874591648e-29,1.01223881828917e-26,1.88280148521559e-29,8.18146543680255e-27,1.6008852719342e-28,3.10794883106975e-26,2.64433215461183e-28,-99.9,-99.9,-99.9,-99.9,-99.9,-99.9,-99.9,-99.9,-99.9,-99.9,-99.9,-99.9,-99.9,-99.9,-99.9,-99.9,2147483645,0.0998
3,1.63808418640755e-28,3.04842579540977e-30,1.70718161598133e-28,4.21233401523035e-30,2.74914071838974e-28,7.18298686171824e-30,3.66638619508692e-28,5.88060153112569e-30,4.58260167982106e-28,5.25004556379856e-30,6.433

In [128]:
example_cat='../../dmu5/dmu5_VIDEO/data/merged/8524/48/8524_48_reducedCat.fits'
example_out='data/8524/48/photoz.in'

In [9]:
#downloaded from 
specz_cat=Table.read('./data/XMM-LSS-specz-v2.91.fits.gz')
specz_cat[:5]

RA,DEC,Z_SPEC,Z_SOURCE,Z_QUAL,REL,OBJID,AGN
float64,float64,float64,int32,int16,bytes5,bytes26,int16
36.4419712,-4.8212023,nan,1,1,0.0,VVDS-J022546.07-044916.3,0
36.6795128,-4.8210224,0.397,1,3,0.975,VVDS-J022643.08-044915.6,0
36.4387748,-4.8203333,1.1714,1,3,0.975,VVDS-J022545.30-044913.1,0
37.0437708,-4.820269,0.9911,1,3,0.975,VVDS-J022810.50-044912.9,0
36.5575475,-4.8206121,1.138,1,2,0.8,VVDS-J022613.81-044914.2,0


In [10]:
t=Table.read(example_cat)
t[:5]

id,VIRCAM_Z_m_base_ClassificationExtendedness_value,VIRCAM_Z_m_base_ClassificationExtendedness_flag,VIRCAM_Z_m_base_CircularApertureFlux_6_0_magErr,VIRCAM_Z_m_base_CircularApertureFlux_6_0_mag,VIRCAM_Z_m_base_CircularApertureFlux_6_0_fluxErr,VIRCAM_Z_m_base_CircularApertureFlux_6_0_flux,VIRCAM_Z_m_base_CircularApertureFlux_6_0_flag,VIRCAM_Y_m_base_ClassificationExtendedness_value,VIRCAM_Y_m_base_ClassificationExtendedness_flag,VIRCAM_Y_m_base_CircularApertureFlux_6_0_magErr,VIRCAM_Y_m_base_CircularApertureFlux_6_0_mag,VIRCAM_Y_m_base_CircularApertureFlux_6_0_fluxErr,VIRCAM_Y_m_base_CircularApertureFlux_6_0_flux,VIRCAM_Y_m_base_CircularApertureFlux_6_0_flag,VIRCAM_Ks_m_merge_peak_sky,VIRCAM_Ks_m_detect_isTractInner,VIRCAM_Ks_m_detect_isPrimary,VIRCAM_Ks_m_detect_isPatchInner,VIRCAM_Ks_m_deblend_nChild,VIRCAM_Ks_m_coord_ra,VIRCAM_Ks_m_coord_dec,VIRCAM_Ks_m_base_ClassificationExtendedness_value,VIRCAM_Ks_m_base_ClassificationExtendedness_flag,VIRCAM_Ks_m_base_CircularApertureFlux_6_0_magErr,VIRCAM_Ks_m_base_CircularApertureFlux_6_0_mag,VIRCAM_Ks_m_base_CircularApertureFlux_6_0_fluxErr,VIRCAM_Ks_m_base_CircularApertureFlux_6_0_flux,VIRCAM_Ks_m_base_CircularApertureFlux_6_0_flag,VIRCAM_J_m_base_ClassificationExtendedness_value,VIRCAM_J_m_base_ClassificationExtendedness_flag,VIRCAM_J_m_base_CircularApertureFlux_6_0_magErr,VIRCAM_J_m_base_CircularApertureFlux_6_0_mag,VIRCAM_J_m_base_CircularApertureFlux_6_0_fluxErr,VIRCAM_J_m_base_CircularApertureFlux_6_0_flux,VIRCAM_J_m_base_CircularApertureFlux_6_0_flag,VIRCAM_H_m_base_ClassificationExtendedness_value,VIRCAM_H_m_base_ClassificationExtendedness_flag,VIRCAM_H_m_base_CircularApertureFlux_6_0_magErr,VIRCAM_H_m_base_CircularApertureFlux_6_0_mag,VIRCAM_H_m_base_CircularApertureFlux_6_0_fluxErr,VIRCAM_H_m_base_CircularApertureFlux_6_0_flux,VIRCAM_H_m_base_CircularApertureFlux_6_0_flag,HSC_Z_m_base_ClassificationExtendedness_value,HSC_Z_m_base_ClassificationExtendedness_flag,HSC_Z_m_base_CircularApertureFlux_6_0_magErr,HSC_Z_m_base_CircularApertureFlux_6_0_mag,HSC_Z_m_base_CircularApertureFlux_6_0_fluxErr,HSC_Z_m_base_CircularApertureFlux_6_0_flux,HSC_Z_m_base_CircularApertureFlux_6_0_flag,HSC_Y_m_base_ClassificationExtendedness_value,HSC_Y_m_base_ClassificationExtendedness_flag,HSC_Y_m_base_CircularApertureFlux_6_0_magErr,HSC_Y_m_base_CircularApertureFlux_6_0_mag,HSC_Y_m_base_CircularApertureFlux_6_0_fluxErr,HSC_Y_m_base_CircularApertureFlux_6_0_flux,HSC_Y_m_base_CircularApertureFlux_6_0_flag,HSC_R_m_coord_ra,HSC_R_m_coord_dec,HSC_R_m_base_ClassificationExtendedness_value,HSC_R_m_base_ClassificationExtendedness_flag,HSC_R_m_base_CircularApertureFlux_6_0_magErr,HSC_R_m_base_CircularApertureFlux_6_0_mag,HSC_R_m_base_CircularApertureFlux_6_0_fluxErr,HSC_R_m_base_CircularApertureFlux_6_0_flux,HSC_R_m_base_CircularApertureFlux_6_0_flag,HSC_I_m_base_ClassificationExtendedness_value,HSC_I_m_base_ClassificationExtendedness_flag,HSC_I_m_base_CircularApertureFlux_6_0_magErr,HSC_I_m_base_CircularApertureFlux_6_0_mag,HSC_I_m_base_CircularApertureFlux_6_0_fluxErr,HSC_I_m_base_CircularApertureFlux_6_0_flux,HSC_I_m_base_CircularApertureFlux_6_0_flag,HSC_G_m_base_ClassificationExtendedness_value,HSC_G_m_base_ClassificationExtendedness_flag,HSC_G_m_base_CircularApertureFlux_6_0_magErr,HSC_G_m_base_CircularApertureFlux_6_0_mag,HSC_G_m_base_CircularApertureFlux_6_0_fluxErr,HSC_G_m_base_CircularApertureFlux_6_0_flux,HSC_G_m_base_CircularApertureFlux_6_0_flag
,,,mag,mag,nJy,nJy,,,,mag,mag,nJy,nJy,,,,,,,deg,deg,,,mag,mag,nJy,nJy,,,,mag,mag,nJy,nJy,,,,mag,mag,nJy,nJy,,,,mag,mag,nJy,nJy,,,,mag,mag,nJy,nJy,,deg,deg,,,mag,mag,nJy,nJy,,,,mag,mag,nJy,nJy,,,,mag,mag,nJy,nJy,
int64,float32,int64,float32,float32,float32,float32,int64,float32,int64,float32,float32,float32,float32,int64,int64,int64,int64,int64,int32,float64,float64,float32,int64,float32,float32,float32,float32,int64,float32,int64,float32,float32,float32,float32,int64,float32,int64,float32,float32,float32,float32,int64,float32,int64,float32,float32,float3

In [11]:
t['ra']=t['VIRCAM_Ks_m_coord_ra']
t['dec']=t['VIRCAM_Ks_m_coord_dec']
for c in specz_cat.colnames:
    specz_cat[c].name=c.lower()
specz_cat['objid'].name='specz_id'
merge= specz_merge(t, specz_cat, radius=1. * u.arcsec)

In [190]:
#From email from Manda
oxfordApCorr = {
    'HSC_G_2as':0.1765,
    'HSC_G_3as':0.0788,
    'HSC_R_2as':0.1638,
    'HSC_R_3as':0.0905,
    'HSC_I_2as':0.1512,
    'HSC_I_3as':0.0788,
    'HSC_Z_2as':0.2155,
    'HSC_Z_3as':0.1144,
    'HSC_Y_2as':0.2023,
    'HSC_Y_3as':0.1144,
    'VIRCAM_Z_2as':0.3567,
    'VIRCAM_Z_3as':0.2023,
    'VIRCAM_Y_2as':0.3567,
    'VIRCAM_Y_3as':0.2023,
    'VIRCAM_J_2as':0.2980,
    'VIRCAM_J_3as':0.1765,
    'VIRCAM_H_2as':0.2423,
    'VIRCAM_H_3as':0.1512,
    'VIRCAM_Ks_2as':0.2288,
    'VIRCAM_Ks_3as':0.1388}

In [220]:
def cat_to_lephare_in(cat_name,write_loc,tract,patch):
    """Take an output patch and make a lephare input file"""
    #Open output cat
    in_cat=Table.read(cat_name)
    #
    
    #Merge the spec-z
    
    out_cat=Table()
    out_cat['Id']=in_cat['id']
    bands='grizyZYJHK'
    n=0
    context=np.full(len(out_cat),0)

    for b in bands:
        physical_filter='{}_{}'.format('VIRCAM' if b.isupper() else 'HSC',b.upper().replace('K','Ks'))
        #mag_col='{}_m_base_CircularApertureFlux_6_0_mag'.format(physical_filter)
        #magerr_col=mag_col+'Err'
        flux_col='{}_m_base_CircularApertureFlux_6_0_flux'.format(physical_filter)
        fluxerr_col=flux_col+'Err'
        lepharename=b #Do we need a different filter name for lephare
        try:
            #Convert to nano jansky and then *(10**-9)*(10**-23)
            #flux,fluxerr=mag_to_flux(in_cat[mag_col],in_cat[magerr_col])
            
            out_cat[physical_filter]=in_cat[flux_col]*(10**-9)*(10**-23)
            out_cat['d_'+physical_filter]=in_cat[fluxerr_col]*(10**-9)*(10**-23)
            #Aperture corrections
            out_cat[physical_filter]*=10**(1*oxfordApCorr['{}_2as'.format(physical_filter)]/2.5)
            out_cat['d_'+physical_filter]*=10**(1*oxfordApCorr['{}_2as'.format(physical_filter)]/2.5)
            mask = np.isnan(out_cat['d_'+physical_filter]) # | (out_cat['d_'+physical_filter]<1e-9) 
            context+=~np.isnan(out_cat['d_'+physical_filter])*2**n
            
            #replace nans with -99.9
            out_cat[physical_filter]=out_cat[physical_filter].astype('float64')
            out_cat['d_'+physical_filter]=out_cat['d_'+physical_filter].astype('float64')
            
            out_cat[physical_filter][mask]=-99.9
            out_cat['d_'+physical_filter][mask]=-99.9
            
            
            n+=1
        except KeyError:
            print('VISTA Error: band {} missing on tract {} and patch {}.'.format(b,tract,patch))
        
        
    out_cat['context']=context
    out_cat['z-spec']=np.full(len(out_cat),-99.9)#np.nan)
    #Write the input catalogue
    Path('data/{}/{}'.format(tract,patch)).mkdir(
                    parents=True, exist_ok=True)
    out_cat.write(write_loc, format='ascii.commented_header',overwrite=True)
    return out_cat

test = cat_to_lephare_in(example_cat,example_out,8524,48)

In [213]:
!ls -al data/8524/48

total 12560
drwxrwxrwx  10 raphaelshirley  staff      320 17 Jan 14:12 .
drwxr-xr-x   4 raphaelshirley  staff      128  6 Dec 15:31 ..
-rw-r--r--@  1 raphaelshirley  staff     6148  3 Jan 13:27 .DS_Store
-rw-r--r--@  1 raphaelshirley  staff     3210 26 Sep 15:20 COSMOS.in.header
-rwxrwxrwx   1 raphaelshirley  staff        0  3 Jan 13:38 output.para
-rw-r--r--@  1 raphaelshirley  staff      271 18 Jan 13:07 photoz.header
-rw-r--r--@  1 raphaelshirley  staff  4414723 18 Jan 16:44 photoz.in
-rw-r--r--@  1 raphaelshirley  staff    11985 17 Jan 15:34 photoz.para
-rw-r--r--   1 raphaelshirley  staff  1117440 13 Jan 15:29 photoz_in.fits
-rw-r--r--@  1 raphaelshirley  staff    12433 18 Jan 15:22 zphot_short.out


In [214]:
test[:5]

Id,HSC_G,d_HSC_G,HSC_R,d_HSC_R,HSC_I,d_HSC_I,HSC_Z,d_HSC_Z,HSC_Y,d_HSC_Y,VIRCAM_Z,d_VIRCAM_Z,VIRCAM_Y,d_VIRCAM_Y,VIRCAM_J,d_VIRCAM_J,VIRCAM_H,d_VIRCAM_H,VIRCAM_Ks,d_VIRCAM_Ks,context,zspec
,nJy,nJy,nJy,nJy,nJy,nJy,nJy,nJy,nJy,nJy,nJy,nJy,nJy,nJy,nJy,nJy,nJy,nJy,nJy,nJy,,
int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,float64
3036815931545223169,2.264631432339382e-30,8.548441772993347e-32,2.2748251312706167e-30,1.588612554013409e-31,2.9249099946199912e-30,2.3555049509404342e-31,3.634417131838461e-30,3.062431736999527e-31,6.525322237798769e-30,9.397898659682864e-31,1.1493553061210186e-30,3.3350903450930646e-43,8.005063866952861e-30,2.1419347896135466e-31,9.98171874356678e-30,3.491207172295296e-31,8.469817845923344e-30,5.2477500264211315e-31,1.088255592409345e-29,7.6159860513787115e-31,1023,-99.9
3036815931545223170,1.0420669963259878e-30,8.657579958248267e-32,1.4531744223904796e-30,1.574206988293517e-31,2.862473000919003e-30,2.4571001064955626e-31,3.176070631514988e-30,3.130902637242834e-31,4.567805913849006e-30,9.853335952547373e-31,1.1467904714870724e-30,3.29305139116332e-43,6.486462839588834e-30,2.138634471674178e-31,6.571137553300034e-30,3.4903610514615743e-31,1.0044345320996669e-29,5.2438944049504344e-31,1.6292208762283845e-29,7.618700973131371e-31,1023,-99.9
3036815931545223172,8.84244466507751e-31,8.674222607267209e-32,1.5722520941683254e-30,1.5679980271324736e-31,1.892391491885959e-30,2.462940667727058e-31,1.7696034897391776e-30,3.13538550249913e-31,2.1020445768974884e-30,9.886915594370703e-31,1.1493523908950286e-30,3.3350903450930646e-43,1.6255271908594567e-30,2.13717920966786e-31,4.921193508064075e-33,3.5085167968088947e-31,3.632901590481833e-30,5.2382106546653385e-31,6.333264140309444e-30,7.6460265148105855e-31,1023,-99.9
3036815931545223173,8.752363805827104e-30,8.43226649080723e-32,1.5609681759484544e-29,1.551335864907308e-31,3.2852326416894107e-29,2.3151262497919483e-31,4.875343598351813e-29,3.0446599081071853e-31,5.688081821911473e-29,9.846401476273003e-31,1.1493523908950286e-30,4.498168070482663e-43,6.695299247549141e-29,2.240140524757514e-31,8.260829319590086e-29,3.6702366076176217e-31,9.875971346998446e-29,5.536969596892928e-31,1.3406052879195774e-28,7.909337249394778e-31,1023,-99.9
3036815931545223174,5.967358045536895e-31,8.436963766233116e-32,1.0622362224369446e-30,1.5592961951016414e-31,1.8437711647449877e-30,2.3725757150019457e-31,2.233774987748941e-30,3.05550266799917e-31,1.126563317014507e-30,9.7094761327073e-31,-99.9,-99.9,-99.9,-99.9,3.1321125970880226e-30,3.992984568359052e-31,5.9052863954801426e-30,6.038946121699713e-31,4.691371999216483e-30,8.737439165311776e-31,927,-99.9


In [47]:
!ls $LEPHAREDIR/source/

LFdata.h                 fit_parameters.cpp       opa.h
Makefile                 fit_schechter.cpp        option_demultipliate.cpp
PDF.cpp                  flt.cpp                  photoz_lib.cpp
PDF.h                    flt.h                    photoz_lib.h
SED.cpp                  globals.cpp              read_LFdata.cpp
SED.h                    globals.h                read_params.h
SEDLib.cpp               keyword.cpp              schechterFCN.cpp
SEDLib.h                 keyword.h                schechterFCN.h
_bindings.cc             mag.cpp                  schechterFunc.cpp
cosmology.cpp            mag.h                    schechterFunc.h
cosmology.h              mag_gal.cpp              sedtolib.cpp
ext.cpp                  minimizationSchech.cpp   split_double.cpp
ext.h                    oneElLambda.cpp          ssfrparaFCN.cpp
ext_curv.cpp             oneElLambda.h            ssfrparaFCN.h
filter.cpp               onesource.cpp            strtodouble.cpp
filter_extinc.cpp  

In [63]:
!touch /Users/raphaelshirley/Documents/github/lsst-ir-fusion/dmu6/dmu6_VIDEO/data/8524/48/output.para

In [86]:
!echo $LEPHAREWORK

/Users/raphaelshirley/Documents/github/LEPHAREWORK


In [92]:
!ls $LEPHAREDIR/bin

CMakeFiles filter     mag_gal    sedtolib   zphota


In [ ]:
#read the filters and compile them into one file stored in $LEPHAREWORK/filt

In [241]:
!$LEPHAREDIR/bin/filter -c data/8524/48/photoz.para

Number of keywords read at the command line (excluding -c config): 0
Reading keywords from data/8524/48/photoz.para
Number of keywords read in the config file: 95
#######################################
# Build the filter file with the following options: 
# Config file: data/8524/48/photoz.para
# FILTER_REP: /Users/raphaelshirley/Documents/github/LEPHARE/filt
# FILTER_LIST: hsc/gHSC.pb hsc/rHSC.pb hsc/iHSC.pb hsc/zHSC.pb hsc/yHSC.pb vista/Z.pb vista/Y.lowres vista/J.lowres vista/H.lowres vista/K.lowres 
# TRANS_TYPE: 1 1 1 1 1 1 1 1 1 1 
# FILTER_CALIB: 0 0 0 0 0 0 0 0 0 0 
# FILTER_FILE: /Users/raphaelshirley/Documents/github/LEPHAREWORK/filt/filter_cosmos.dat
# FILTER_FILE.doc: /Users/raphaelshirley/Documents/github/LEPHAREWORK/filt/filter_cosmos.doc
#######################################
# NAME                        IDENT      Lbda_mean Lbeff(Vega)       FWHM     AB-cor    TG-cor      VEGA M_sun(AB)   CALIB      Lb_eff    Fac_corr
hsc/gHSC.pb                   1             0.4816

In [242]:
!ls -al $LEPHAREWORK/filt

total 400
drwxr-xr-x  4 raphaelshirley  staff     128 18 Jan 14:17 .
drwxr-xr-x  6 raphaelshirley  staff     192 18 Jan 14:16 ..
-rw-r--r--@ 1 raphaelshirley  staff  164566 18 Jan 17:18 filter_cosmos.dat
-rw-r--r--@ 1 raphaelshirley  staff    1349 18 Jan 17:18 filter_cosmos.doc


In [ ]:
#This file must exist to be written to
!touch /Users/raphaelshirley/Documents/github/lsst-ir-fusion/dmu6/dmu6_VIDEO/data/8524/48/output.para

In [ ]:
#read the galaxy templates (as used in Ilbert et al. 2013)and store them in $LEPHAREWORK/lib_bin

In [ ]:
!touch COSMOS_MOD.list

In [150]:
!ls -al

total 58472
drwxr-xr-x  17 raphaelshirley  staff       544 17 Jan 14:48 .
drwxr-xr-x   8 raphaelshirley  staff       256  7 Dec 14:38 ..
-rw-r--r--@  1 raphaelshirley  staff      8196  3 Jan 13:26 .DS_Store
drwxr-xr-x   8 raphaelshirley  staff       256  3 Jan 13:25 .ipynb_checkpoints
-rw-r--r--@  1 raphaelshirley  staff     54512  2 Aug 13:09 0_Basic_table operations.ipynb
-rw-r--r--@  1 raphaelshirley  staff    809846  2 Sep 15:45 1_Make_sxds_photoz_input.ipynb
-rw-r--r--   1 raphaelshirley  staff    183094 17 Jan 14:48 2_Run_lephare.ipynb
-rw-r--r--   1 raphaelshirley  staff         0 17 Jan 14:44 COSMOS_MOD.list
drwxr-xr-x@ 66 raphaelshirley  staff      2112 14 Nov 11:49 data
drwxr-xr-x   5 raphaelshirley  staff       160  3 Aug 13:51 figs
drwxr-xr-x  22 raphaelshirley  staff       704 12 Oct  2021 filters
-rw-r--r--   1 raphaelshirley  staff  25397625 11 Jul  2022 lsst_ir_fusion_sxds_photoz_input_specz_sample_20220704.dat
-rw-r--r--   1 raphaelshirley  staff    658979 14 Dec 16:41

In [244]:
!$LEPHAREDIR/bin/sedtolib -c data/8524/48/photoz.para -t G -GAL_SED $LEPHAREDIR/examples/COSMOS_MOD.list  -GAL_LIB LIB_VISTA

Number of keywords read at the command line (excluding -c config): 3
Reading keywords from data/8524/48/photoz.para
Number of keywords read in the config file: 95
Keyword SEL_AGE not provided 
#######################################
# It s translating SEDs to binary library #
# with the following options :           
# Config file     : data/8524/48/photoz.para
# Library type     : GAL
# GAL_SED    :COSMOS_MOD.list
# GAL_LIB    :LIB_VISTA
# GAL_LIB doc:/Users/raphaelshirley/Documents/github/LEPHAREWORK/lib_bin/LIB_VISTA.doc
# GAL_LIB phys:/Users/raphaelshirley/Documents/github/LEPHAREWORK/lib_bin/LIB_VISTA.phys
# SEL_AGE    :none
# GAL_FSCALE :1
# AGE_RANGE   0 1.5e+10
#######################################
Number of SED in the list 31

Times to run the code in sec: 0.096428


In [ ]:
#use the galaxy templates + filters to derive a library of predicted magnitudes and store 
#it in $WORK/lib_mag (the parameters correspond to enabling emission lines correlated to UV light + 
#free factor in scaling these lines, more information in LePhare_documentation.pdf)

In [249]:
#This may need to be run at the terminal due to a data rate limit
%%bash
$LEPHAREDIR/bin/mag_gal  \
    -c data/8524/48/photoz.para -t G -GAL_LIB_IN LIB_VISTA -GAL_LIB_OUT VISTA_COSMOS_FREE \
    -MOD_EXTINC 18,26,26,33,26,33,26,33 \
    -EXTINC_LAW SMC_prevot.dat,SB_calzetti.dat,SB_calzetti_bump1.dat,SB_calzetti_bump2.dat \
    -EM_LINES EMP_UV  -EM_DISPERSION 0.5,0.75,1.,1.5,2. -Z_STEP 0.04,0,6

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [ ]:
#proceed in the same way for stellar and QSO templates

In [261]:
%%bash
$LEPHAREDIR/bin/sedtolib -c data/8524/48/photoz.para -t S -STAR_SED $LEPHAREDIR/examples/STAR_MOD_ALL.list
$LEPHAREDIR/bin/mag_gal -c data/8524/48/photoz.para -t S -LIB_ASCII YES -STAR_LIB_OUT ALLSTAR_COSMOS 
# AGN models from Salvato 2009
$LEPHAREDIR/bin/sedtolib -c data/8524/48/photoz.para -t Q -QSO_SED  $LEPHAREDIR/sed/QSO/SALVATO09/AGN_MOD.list
$LEPHAREDIR/bin/mag_gal -c data/8524/48/photoz.para -t Q -MOD_EXTINC 0,1000  -EB_V 0.,0.1,0.2,0.3 \
    -EXTINC_LAW SB_calzetti.dat -LIB_ASCII NO  -Z_STEP 0.04,0,6


Number of keywords read at the command line (excluding -c config): 2
Reading keywords from data/8524/48/photoz.para
Number of keywords read in the config file: 95
Keyword SEL_AGE not provided 
#######################################
# It s translating SEDs to binary library #
# with the following options :           
# Config file     : data/8524/48/photoz.para
# Library type     : STAR
# STAR_SED    :/Users/raphaelshirley/Documents/github/LEPHARE/examples/STAR_MOD_ALL.list
# STAR_LIB    :LIB_STAR
# STAR_LIB doc:/Users/raphaelshirley/Documents/github/LEPHAREWORK/lib_bin/LIB_STAR.doc
# STAR_FSCALE :3.432e-09
#######################################
Number of SED in the list 254

Times to run the code in sec: 1.01563
Running with OpenMP on 4 threads!
Number of keywords read at the command line (excluding -c config): 3
Reading keywords from data/8524/48/photoz.para
Number of keywords read in the config file: 95
 All files opened 
#######################################
# It s computing the

In [263]:
!ls -al $LEPHAREWORK/lib_mag

total 700888
drwxr-xr-x  9 raphaelshirley  staff        288  3 Oct 14:43 .
drwxr-xr-x  6 raphaelshirley  staff        192 18 Jan 14:16 ..
-rw-r--r--  1 raphaelshirley  staff   18053280 19 Jan 13:08 ALLSTAR_COSMOS.bin
-rw-r--r--  1 raphaelshirley  staff      29946 19 Jan 13:08 ALLSTAR_COSMOS.dat
-rw-r--r--  1 raphaelshirley  staff        642 19 Jan 13:08 ALLSTAR_COSMOS.doc
-rw-r--r--  1 raphaelshirley  staff    3284064 19 Jan 13:08 QSO_COSMOS.bin
-rw-r--r--  1 raphaelshirley  staff        615 19 Jan 13:08 QSO_COSMOS.doc
-rw-r--r--  1 raphaelshirley  staff  334060800 19 Jan 12:52 VISTA_COSMOS_FREE.bin
-rw-r--r--  1 raphaelshirley  staff        732 19 Jan 12:52 VISTA_COSMOS_FREE.doc


In [276]:
!ls -al $LEPHAREWORK/lib_bin

total 39776
drwxr-xr-x  9 raphaelshirley  staff       288  3 Oct 14:42 .
drwxr-xr-x  6 raphaelshirley  staff       192 18 Jan 14:16 ..
-rw-r--r--  1 raphaelshirley  staff    398608 19 Jan 13:08 LIB_QSO.bin
-rw-r--r--  1 raphaelshirley  staff      1915 19 Jan 13:08 LIB_QSO.doc
-rw-r--r--  1 raphaelshirley  staff  18037024 19 Jan 13:08 LIB_STAR.bin
-rw-r--r--  1 raphaelshirley  staff     28957 19 Jan 13:08 LIB_STAR.doc
-rw-r--r--  1 raphaelshirley  staff    878168 18 Jan 17:21 LIB_VISTA.bin
-rw-r--r--  1 raphaelshirley  staff      3624 18 Jan 17:21 LIB_VISTA.doc
-rw-r--r--  1 raphaelshirley  staff      4867 18 Jan 17:21 LIB_VISTA.phys


In [277]:
!ls -al $LEPHAREWORK/filt

total 400
drwxr-xr-x  4 raphaelshirley  staff     128 18 Jan 14:17 .
drwxr-xr-x  6 raphaelshirley  staff     192 18 Jan 14:16 ..
-rw-r--r--@ 1 raphaelshirley  staff  164566 18 Jan 17:18 filter_cosmos.dat
-rw-r--r--@ 1 raphaelshirley  staff    1349 18 Jan 17:18 filter_cosmos.doc


In [274]:
!ls -al $LEPHAREWORK

total 16
drwxr-xr-x   6 raphaelshirley  staff   192 18 Jan 14:16 .
drwxr-xr-x  37 raphaelshirley  staff  1184 20 Sep 11:43 ..
-rw-r--r--@  1 raphaelshirley  staff  6148 18 Jan 18:46 .DS_Store
drwxr-xr-x   4 raphaelshirley  staff   128 18 Jan 14:17 filt
drwxr-xr-x   9 raphaelshirley  staff   288  3 Oct 14:42 lib_bin
drwxr-xr-x   9 raphaelshirley  staff   288  3 Oct 14:43 lib_mag


In [278]:
%%bash --out t
$LEPHAREDIR/bin/zphota \
    -c data/8524/48/photoz.para \
    -CAT_IN data/8524/48/photoz.in \
    -CAT_OUT data/8524/48/zphot_short.out \
    -PARA_OUT data/8524/48/output.para \
    -ZPHOTLIB VISTA_COSMOS_FREE,ALLSTAR_COSMOS,QSO_COSMOS \
    -ADD_EMLINES 0,100 \
    -AUTO_ADAPT NO \
    -Z_STEP 0.04,0,6

In [279]:
#t

In [280]:
!ls -al data/8524/48/

total 43264
drwxrwxrwx  10 raphaelshirley  staff       320 17 Jan 14:12 .
drwxr-xr-x   4 raphaelshirley  staff       128  6 Dec 15:31 ..
-rw-r--r--@  1 raphaelshirley  staff      6148  3 Jan 13:27 .DS_Store
-rw-r--r--@  1 raphaelshirley  staff      3210 26 Sep 15:20 COSMOS.in.header
-rwxrwxrwx@  1 raphaelshirley  staff      1847 19 Jan 13:35 output.para
-rw-r--r--@  1 raphaelshirley  staff       272 18 Jan 17:15 photoz.header
-rw-r--r--@  1 raphaelshirley  staff   4414724 18 Jan 17:13 photoz.in
-rw-r--r--@  1 raphaelshirley  staff     12108 19 Jan 13:28 photoz.para
-rw-r--r--   1 raphaelshirley  staff   1117440 13 Jan 15:29 photoz_in.fits
-rw-r--r--@  1 raphaelshirley  staff  15170817 19 Jan 13:40 zphot_short.out


In [281]:
with open('./data/8524/48/output_{}.txt'.format(time.strftime("%Y%m%d")),'w+') as f:
    f.write(t)
f.close()

In [282]:
print(t[0:30000])


Starting times in sec: 0.011886
local:     Thu Jan 19 13:35:29 2023

Number of keywords read at the command line (excluding -c config): 7
Reading keywords from data/8524/48/photoz.para
Number of keywords read in the config file: 95
Keyword MASS_SCALE not provided 
Keyword NZ_PRIOR not provided 
Keyword RM_DISCREPENT_BD not provided 
Keyword PROB_INTZ not provided 
Keyword M_REF not provided 
Keyword APPLY_SYSSHIFT not provided 
Keyword PDZ_TYPE not provided 
####################################### 
# PHOTOMETRIC REDSHIFT with OPTIONS   # 
# Config file            : data/8524/48/photoz.para
# CAT_IN                 : data/8524/48/photoz.in
# CAT_OUT                : data/8524/48/zphot_short.out
# CAT_LINES              : 0 1000000000
# PARA_OUT               : data/8524/48/output.para
# INP_TYPE               : F
# CAT_FMT[0:MEME 1:MMEE] : 0
# CAT_MAG                : AB
# ZPHOTLIB               : VISTA_COSMOS_FREE ALLSTAR_COSMOS QSO_COSMOS 
# FIR_LIB                : 
# FIR_LMIN      

In [257]:
!ls -al data/8524/48/

total 12560
drwxrwxrwx  10 raphaelshirley  staff      320 17 Jan 14:12 .
drwxr-xr-x   4 raphaelshirley  staff      128  6 Dec 15:31 ..
-rw-r--r--@  1 raphaelshirley  staff     6148  3 Jan 13:27 .DS_Store
-rw-r--r--@  1 raphaelshirley  staff     3210 26 Sep 15:20 COSMOS.in.header
-rwxrwxrwx   1 raphaelshirley  staff        0  3 Jan 13:38 output.para
-rw-r--r--@  1 raphaelshirley  staff      272 18 Jan 17:15 photoz.header
-rw-r--r--@  1 raphaelshirley  staff  4414724 18 Jan 17:13 photoz.in
-rw-r--r--@  1 raphaelshirley  staff    12021 18 Jan 17:17 photoz.para
-rw-r--r--   1 raphaelshirley  staff  1117440 13 Jan 15:29 photoz_in.fits
-rw-r--r--@  1 raphaelshirley  staff    12431 19 Jan 13:00 zphot_short.out


In [283]:
out=Table.read('./data/8524/48/zphot_short.out',format='ascii.csv')

In [284]:
out[:11]

# Creation date: Thu Jan 19 13:35:29 2023
str1430
#######################################
# PHOTOMETRIC REDSHIFT with OPTIONS #
# Config file : data/8524/48/photoz.para
# CAT_IN : data/8524/48/photoz.in
# CAT_OUT : data/8524/48/zphot_short.out
# CAT_LINES : 0 1000000000
# PARA_OUT : data/8524/48/output.para
# INP_TYPE : F
# CAT_FMT[0:MEME 1:MMEE] : 0


In [285]:
out

# Creation date: Thu Jan 19 13:35:29 2023
str1430
#######################################
# PHOTOMETRIC REDSHIFT with OPTIONS #
# Config file : data/8524/48/photoz.para
# CAT_IN : data/8524/48/photoz.in
# CAT_OUT : data/8524/48/zphot_short.out
# CAT_LINES : 0 1000000000
# PARA_OUT : data/8524/48/output.para
# INP_TYPE : F
# CAT_FMT[0:MEME 1:MMEE] : 0


In [240]:
#For header file
n=1
for t in test.colnames:
    print(n, "${}".format(t))
    n+=1

1 $Id
2 $HSC_G
3 $d_HSC_G
4 $HSC_R
5 $d_HSC_R
6 $HSC_I
7 $d_HSC_I
8 $HSC_Z
9 $d_HSC_Z
10 $HSC_Y
11 $d_HSC_Y
12 $VIRCAM_Z
13 $d_VIRCAM_Z
14 $VIRCAM_Y
15 $d_VIRCAM_Y
16 $VIRCAM_J
17 $d_VIRCAM_J
18 $VIRCAM_H
19 $d_VIRCAM_H
20 $VIRCAM_Ks
21 $d_VIRCAM_Ks
22 $context
23 $z-spec
